In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv("billets.csv")
data = pd.DataFrame(data, columns=data.columns)

data.head()

,is_genuine,diagonal,height_left,height_right,margin_low,margin_up,length
0,True,171.81,104.86,104.95,4.52,2.89,112.83
1,True,171.67,103.74,103.70,4.01,2.87,113.29
2,True,171.83,103.76,103.76,4.40,2.88,113.84
3,True,171.80,103.78,103.65,3.73,3.12,113.63
4,True,172.05,103.70,103.75,5.04,2.27,113.55


# Partie 1 : ACP

In [3]:
from functions2 import *
from sklearn import preprocessing, decomposition

## Composantes principales

In [4]:
n_comp = 5

data_pca = data[["diagonal", "height_left", "height_right", "margin_low", "margin_up", "length"]]

data_pca = data_pca.fillna(data_pca.mean())

#On ne prend que les valeurs
X = data_pca.values

#La variable qui nous permettra de colorer nos résultats : 
fakeornot = [data.loc[fake, "is_genuine"] for fake in data.index]

names = data_pca.index

features = data_pca.columns

#Centrage et réduction : 
std_scale = preprocessing.StandardScaler().fit(X)
X_scaled = std_scale.transform(X)

#Calcul des composantes principales : 
pca = decomposition.PCA(n_components = n_comp)
pca = pca.fit(X_scaled)

# Mission 3 : Regression Logistique

In [5]:
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [6]:
data['is_genuine'] = list(map(int, data['is_genuine'])) 

In [7]:
#On est obligé d'enlever Length sinon message d'erreur
reg_log1 = smf.glm('is_genuine~margin_low+margin_up',
                   data=data,
                   family=sm.families.Binomial()).fit()
print(reg_log1.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:             is_genuine   No. Observations:                  170
Model:                            GLM   Df Residuals:                      167
Model Family:                Binomial   Df Model:                            2
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -6.3603
Date:                Mon, 20 May 2019   Deviance:                       12.721
Time:                        16:25:02   Pearson chi2:                     11.3
No. Iterations:                    11   Covariance Type:             nonrobust
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    176.4638     74.203      2.378      0.017      31.029     321.898
margin_low   -16.5356      6.923     -2.388      0.0

# Programme :

In [47]:
def billets (df):
    #On récupère les noms de colonnes :
    columns_df = list(df.columns)
    columns_data = list(data[['diagonal','height_left', 
                          'height_right','margin_low', 
                          'margin_up','length']].columns)
    
    #On check si une colonne id existe pour l'utiliser en index par la suite : 
    if any("id" in i for i in columns_df):
        id_df = df['id']
    else:
        id_df = df.index    
    
    #On les compare avec celles de notre fichier de train
    result =  all(elem in columns_df  for elem in columns_data)

    if result == True :
        pass
    else:
        print("Les données fournies ne contiennent pas les colonnes requises.")
        print("Pour que le programme fonctionne, il faut les colonnes suivantes :")
        print(columns_data)
        return None
    
    #On gère les valeurs manquantes :
    df = df.fillna(df.mean()) 
    
    #On prend les colonnes utiles pour le centrage/réduction : 
    df_cr = df[["diagonal", "height_left", 
                 "height_right", "margin_low", 
                 "margin_up", "length"]]
    
    #On applique la prédiction de la régression logistique :
    df_predict = pd.DataFrame(df_cr, index=id_df, columns=df_cr.columns)
    df_predict = reg_log1.predict(df_cr)
    
    #On modifie le df retourné pour avoir l'identifiant, la proba et True or False :
    df_predict = pd.DataFrame(df_predict, index=df.index, columns=["Proba"])
    df_predict['Proba'] = df_predict['Proba']*100
    df_predict["is_genuine"] = np.where(df_predict["Proba"] >= 50, True, False)
    
    #On enregistre le fichier dans un répertoire :
    #df_final.to_csv("previsions.csv")
    
    #On calcule le nombre de True et de False : 
    true = df_predict[df_predict["is_genuine"] == True].count()
    false = df_predict[df_predict["is_genuine"] == False].count()
    
    #On print le tout : 
    print("Le nombre de True est de : ", true[1])
    print("Le nombre de False est de : ", false[1])
    
    #On save le fichier avec les True uniquement : 
    vrais_billets = df_predict[df_predict["is_genuine"] == True]
    #vrais_billets.to_csv("vrais_billets.csv")
    
    #On return le tout pour l'avoir en sortie de programme :
    return df_predict

## Matrice de confusion et accuracy

In [38]:
#Préparation des données : 
test_data = data.iloc[:,:]
test_data = billets(test_data)
test_data = test_data['is_genuine']

data_confusion1 = data.copy()
data_confusion1["y_pred"] = test_data

#Calcul de la matrice de confusion :
from sklearn.metrics import confusion_matrix
conf_matrix1 = confusion_matrix(data_confusion1["is_genuine"], 
                               data_confusion1['y_pred'])

conf_df1 = {"Vrai":[conf_matrix1[1,1],conf_matrix1[0,1]], 
           "Faux":[conf_matrix1[0,0],conf_matrix1[1,0]]}
conf_df1 = pd.DataFrame(conf_df1, index=[["Positif", "Négatif"]])

print(conf_df1)
print("\n")
print("L'accuracy est de : ",
      round((conf_matrix1[1,1]+conf_matrix1[0,0])/len(data_confusion1)*100))

Le nombre de True est de :  101
Le nombre de False est de :  69
         Vrai  Faux
Positif    99    68
Négatif     2     1


L'accuracy est de :  98.0


<p> Accuracy de 98% </p>

In [48]:
test = pd.read_csv("example.csv")
billets(test)

Le nombre de True est de :  2
Le nombre de False est de :  3


,Proba,is_genuine
0,6.776884e-05,False
1,1.047737e-10,False
2,1.489232e-04,False
3,9.991583e+01,True
4,9.999992e+01,True
